# Error

In [7]:
import numpy as np
import pandas as pd


In [10]:
def calcular_metricas_erro(y_true, y_pred, naive_forecast=None):
    """
    Parâmetros:
        y_true (array): Valores reais observados.
        y_pred (array): Valores previstos pelo modelo.
        naive_forecast (array, opcional): Previsões do modelo naive (para MASE).
                                         Se None, será calculado internamente.

    Retorna:
        dict: Dicionário com as métricas calculadas.
    """

    # Converte para arrays numpy para evitar problemas com Pandas/Series
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)

    # Garante que os arrays têm o mesmo comprimento
    if len(y_true) != len(y_pred):
        raise ValueError("y_true e y_pred devem ter o mesmo comprimento.")

    # Cálculo dos erros individuais
    errors = y_true - y_pred
    absolute_errors = np.abs(errors)
    squared_errors = errors ** 2
    percentage_errors = np.abs(errors) / np.abs(y_true)

    # MAE (Mean Absolute Error) - Erro Absoluto Médio
    mae = np.mean(absolute_errors)

    # MSE (Mean Squared Error) - Erro Quadrático Médio
    mse = np.mean(squared_errors)

    # RMSE (Root Mean Squared Error) - Raiz do Erro Quadrático Médio
    rmse = np.sqrt(mse)

    # MAPE (Mean Absolute Percentage Error) - Erro Percentual Absoluto Médio
    # Pequeno epsilon para evitar divisão por zero
    y_true_non_zero = np.where(y_true == 0, 1e-10, y_true) 
    safe_percentage_errors = np.abs(errors) / np.abs(y_true_non_zero)
    # Convertendo para percentual
    mape = np.mean(safe_percentage_errors) * 100 

    # MASE (Mean Absolute Scaled Error) - Erro Absoluto Médio Escalonado
    if naive_forecast is None:
        # Calcula o modelo naive: y_naive[t] = y_true[t-1]
        naive_forecast = np.roll(y_true, 1)
        # A primeira observação não tem um valor anterior, então descarta-se
        naive_forecast[0] = np.nan

    # Calcula o erro absoluto médio do modelo naive (denominador do MASE)
    naive_errors = y_true - naive_forecast
    naive_mae = np.nanmean(np.abs(naive_errors))

    # Evita divisão por zero no MASE
    if naive_mae == 0:
        mase = np.inf # Se o modelo naive for perfeito, MASE é infinito
    else:
        mase = np.mean(absolute_errors) / naive_mae

    return {
        'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape, 'MASE': mase
    }

In [11]:
# --- EXEMPLO PRÁTICO ---

# Dados de exemplo (substitua pelos seus dados reais e previsões)
dados_reais = np.array([120, 135, 148, 165, 180])
previsoes_modelo = np.array([118, 130, 152, 160, 175])

# Calcula as métricas
metricas = calcular_metricas_erro(dados_reais, previsoes_modelo)

# Exibe os resultados de forma organizada
print("Métricas de Erro do Modelo de Previsão:")
for nome, valor in metricas.items():
    print(f"{nome}: {valor:.4f}")

Métricas de Erro do Modelo de Previsão:
MAE: 4.2000
MSE: 19.0000
RMSE: 4.3589
MAPE: 2.7762
MASE: 0.2800
